In [1]:
import os
import sys
import pathlib
import platform
from copy import deepcopy

from typing import (
    Dict,
    List
)

In [2]:
# CLI
mod_path: str = os.path.join(str(pathlib.Path(os.path.abspath(os.getcwd())).parents[1]))
sys.path.append(mod_path)

In [3]:
from convert_source.cs_utils.fileio import (
    ConversionError,
    Command
)

In [4]:
from convert_source.cs_utils.utils import (
    SubDataInfo,
    list_dict,
    comp_dict,
    depth,
    convert_image_data,
    file_to_screen,
    collect_info,
    get_metadata
)

In [5]:
from convert_source.batch_convert import (
    BIDSImgData,
    read_config,
    bids_id,
    _gather_bids_name_args,
    _get_bids_name_args,
    make_bids_name,
    data_to_bids,
    batch_proc,
    source_to_bids
)

In [6]:
from convert_source.cs_utils.const import BIDS_PARAM

In [7]:
scripts_dir: str = os.path.join(os.getcwd(),'helper.scripts')
test_config: str = os.path.join(scripts_dir,'test.config.yml')

data_dir: str = os.path.join(os.getcwd(),'test.study_dir')
dcm_test_data: str = os.path.join(data_dir,'TEST001-UNIT001','data.dicom','ST000000')

In [8]:
# dcm2niix should already be downloaded
dcm2niix_cmd: Command = Command("dcm2niix")
dcm2niix_cmd.check_dependency(path_envs=[os.getcwd()])

True

In [9]:
verbose: bool = True
[search_dict,
bids_search,
bids_map,
meta_dict,
exclusion_list] = read_config(config_file=test_config,
                            verbose=verbose)

Initialized parameters from configuration file
Categorizing search terms
Including BIDS related search term settings
Corresponding BIDS mapping settings
Including additional settings for metadata
Exclusion option implemented


In [10]:

subs_data: List[SubDataInfo] = collect_info(parent_dir=data_dir,exclusion_list=[])

In [11]:
len(subs_data)

14

In [12]:
for i in range(0,len(subs_data)):
    if '.nii' in subs_data[i].data.lower():
        print(f"{i} : {subs_data[i].data}")

3 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/DWI_68_DIR.nii.gz
4 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/DWI_B0.nii.gz
5 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/FLAIR.nii.gz
6 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/T1_AXIAL.nii.gz
7 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/T2_InPlane.nii.gz
8 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/rs_fMRI.nii.gz
9 : /Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/rsfMRI.nii.gz


In [13]:
# This one throws error
subs_data[0].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000000/MR000002.dcm'

In [14]:
# dcm test
subs_data[1].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.dicom/ST000000/SE000001/MR000024.dcm'

In [15]:
# par test - NOTE: no corresponding image data
subs_data[11].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.parrec/AX_SWIP_MPR.PAR'

In [16]:
# nii tests
subs_data[9].data

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.study_dir/TEST001-UNIT001/data.nifti/rsfMRI.nii.gz'

In [24]:
bids_name_dict: Dict = deepcopy(BIDS_PARAM)
bids_name_dict['info']['sub'] = subs_data[0].sub
if subs_data[0].ses:
    bids_name_dict['info']['ses'] = subs_data[0].ses

[bids_name_dict, 
 modality_type, 
 modality_label, 
 task] = bids_id(s=subs_data[0].data,
                 search_dict=search_dict,
                 bids_search=bids_search,
                 bids_map=bids_map,
                 bids_name_dict=bids_name_dict,
                 parent_dir=data_dir)
print(modality_type)
print(modality_label)
print(task)

anat
T1w



In [25]:
[meta_com_dict, 
meta_scan_dict] = get_metadata(dictionary=meta_dict,
                            modality_type=modality_type,
                            task=task)

In [26]:
meta_com_dict

{'Manufacturer': 'Vendor',
 'ManufacturersModelName': 'Scanner model',
 'MagneticFieldStrength': 3,
 'InstitutionName': 'Institution Name'}

In [27]:
meta_scan_dict

{}

In [28]:
out_dir: str = os.path.join(os.getcwd(),'test.bids')

In [29]:
out_dir

'/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids'

In [30]:
[imgs,
jsons,
bvals,
bvecs] = data_to_bids(sub_data=subs_data[0],
                    bids_name_dict=bids_name_dict,
                    out_dir=out_dir,
                    modality_type=modality_type,
                    modality_label=modality_label,
                    task=task,
                    meta_dict=meta_com_dict,
                    mod_dict=meta_scan_dict)
# data_to_bids(sub_data=subs_data[1],
#                     bids_name_dict=bids_name_dict,
#                     out_dir=out_dir,
#                     modality_type=modality_type,
#                     modality_label=modality_label,
#                     task=task,
#                     meta_dict=meta_com_dict,
#                     mod_dict=meta_scan_dict)

/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/tmp_dir_7889


In [31]:
imgs

['']

In [23]:
source_to_bids(sub_data=subs_data[1],
                    bids_name_dict=bids_name_dict,
                    out_dir=out_dir,
                    modality_type=modality_type,
                    modality_label=modality_label,
                    task=task,
                    meta_dict=meta_com_dict,
                    mod_dict=meta_scan_dict)

/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/tmp_dir_2521
ERROR: Error: Missing images. Found 8 images, expected 27 slices per volume and instance number (0020,0013) ranges from 2 to 25



(['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-01_T2w.nii.gz',
  '/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-01_T2w.nii.gz'],
 ['/Users/adebayobraimah/Desktop/projects/convert_source/convert_source/tests/test.bids/sub-TEST001/ses-UNIT001/anat/sub-TEST001_ses-UNIT001_run-01_T2w.json',
  ''],
 ['', ''],
 ['', ''])